# 1: Base Data Inspection

Feel free to minimise from Base Data Inspection, I got lost down rabbit holes before I realised this was a machine learning subject not feature engineering. 

That said I found there was ~100,000 duplicate rows, 0 duplicated columns, 0 constant columns. 

Chose not to remove the duplicate rows as, I was to *consider that the amount of data for each species in the database available is an indication of its abundance or rarity*

Making it a Hierarchal Multiclassification, edge detection problem, given the scarcity of specific wood types, the target to inditify would be less likely to have duplicates then others, resulting in more complicated classification in this scenario.

In [ ]:
def squish(INS: Path, OUT: Path, CLASS_HEADERS: list[str]):      
    files = [
"Data\BGLBP.parquet",
"Data\CSLBP.parquet",
"Data\CSSILTP.parquet",
"Data\OLBP.parquet",
"Data\SCSLBP.parquet",
"Data\SILTP.parquet",
"Data\Tchebyshev.parquet",
]
    dfs = [pl.read_parquet(f) for f in files]
    master = dfs[0]
    for df in dfs[1:]:
        feat_cols = [c for c in df.columns if c.lower() not in CLASS_HEADERS]
        master = master.hstack(df.select(feat_cols))
    master.write_parquet(OUT)

In [ ]:
import polars as pl
for file in RAW.glob("*.csv"):
    name = file.stem.split("_")[1]
    print(name)
    df = pl.read_csv(file, has_header = False)
    ncols = len(df.columns) 
    new_headers = CLASS_HEADERS + [f"{name}_{i}" for i in range(1, ncols - len(CLASS_HEADERS) + 1)]
    df = df.rename(dict(zip(df.columns, new_headers)))
    df.write_parquet(BASE / f"{name}.parquet")
    print(f"Rows x Cols: {len(df)} x {ncols}")

In [1]:
from pathlib import Path
BASE = Path("Data")
RAW = BASE / "Raw"
MASTER = BASE / "master.parquet"

In [ ]:
CLASS_HEADERS = ["family", "genus", "species"]
squish(BASE, MASTER, CLASS_HEADERS)
data = pd.read_csv(MASTER)
data.describe()

### Prelim Analysis

In [ ]:
FIGURESIZE = (10,6)
FONTSIZE = 18
newdata = data.copy()
newdata["index"] = range(1, len(newdata) + 1)
np_data = newdata.loc[:,["family", "genus", "species", "index"]].to_numpy()
family, genus, species, index = np_data[:,0], np_data[:,1], np_data[:,2], np_data[:, 3]
nind = len(newdata["index"])

plt.figure(figsize=FIGURESIZE)
plt.plot(index, family, "r", label="Family")
plt.plot(index, genus, "g", label="Genus")
plt.plot(index, species, "b", label="Species")
plt.fill_between(index, family, 0, color="r", alpha=0.3)
plt.fill_between(index, genus, family, color="g", alpha=0.3)
plt.fill_between(index, species, genus, color="b", alpha=0.3)
plt.ylabel("Instance as Table Index", fontsize=FONTSIZE)
plt.xlabel("Index", fontsize=FONTSIZE)
plt.legend()
plt.tight_layout()
plt.show()

NameError: name 'data' is not defined

In [ ]:
plt.figure(figsize=FIGURESIZE)
plt.plot(index, family, "r", label="Family")
plt.plot(index, genus, "g", label="Genus")
plt.plot(index, species, "b", label="Species")
plt.xlim(0, nind)
plt.ylim(-10, 70)
plt.fill_between(index, family, 0, color="r", alpha=0.3)
plt.fill_between(index, genus, family, color="g", alpha=0.3)
plt.fill_between(index, species, genus, color="b", alpha=0.3)
plt.ylabel("Instance as Table Index", fontsize=FONTSIZE)
plt.xlabel("Index", fontsize=FONTSIZE)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
for file in files:
    df = pd.read_parquet(file)
    print(file)
    matrix = df.corr()
    classes = ["family", "genus", "species"]
    features = [c for c in df.columns if c not in classes]
    df.hist(column = features, xlabelsize = 5, ylabelsize = 5, figsize = (10,6), bins = 21)
    plt.figure(figsize=(40,40))
    sns.heatmap(matrix, annot = True, cmap = "coolwarm", fmt = ".2f", linewidths = 0.5)
    plt.title(f"Correlation Matrix: {file}")

In [ ]:
df = data.copy()
classes = ["family", "genus", "species"]
features = [col for col in df.columns if col not in classes]

In [ ]:
for c in classes:
    min, max = df[c].min(), df[c].max()
    exp = set(range(min, max + 1))
    present = set(df[c].unique())
    missing = sorted(exp - present)
    unique = len(df[c].unique())
    print("_____________________________________________________________________")
    print(c)
    print(f"Uniques {unique}, Minimum: {min}, Maximum: {max}, Missing: {missing}") 

In [ ]:
for feature in features:
    dracula = len(df[feature].unique())
    if dracula <= 2:
        print(f"{feature}: {dracula}")
    else:
        None

### Base Data Inspection: Key-Takeaways

The labels, whilst being numerical, are just labels, so the line plots might give an understanding of frequency per class pair offers limited insight into the data being presented.

DataFrame Dimensions: 293830 x 536 (R x C)

Constant Columns: None / All columns had at least 2 Unique values

* Class Histograms:
    - Families:
        - 58 Unique
        - 7 familily groups (groups being ~ 3 familiys) saw (less) than 5_000 observations. 
        - 5 familily groups (groups being ~ 3 familiys) saw (more) than 25_000 observations. 
        - Missing families [1, 28]
    - Genus:
        - 191 Unique
        - 8 Genus groups (groups being ~ 9 Geni) saw (less) than 10_000 observations. 
        - 4 Genus groups (groups being ~ 9 Geni) saw (more) than 20_000 observations. 
        - Missing Genus [30, 62, 70, 110, 141]
    - Species:
        - 925 Unique
        - 9 Species groups (groups being ~ 44 Species) saw (less) than 10_000 observations. 
        - 6 Species groups (groups being ~ 44 Species) saw (more) than 20_000 observations.
        - Missing species [114, 229] 

Classes & features are both very skewed and  hierarchal (family -> genus -> species) and multi-class (classes are non-binary) in nature

In [ ]:
def distribution_vs_expected(df: pl.DataFrame, classes: str):
    exp_even = len(df) / df.select(pl.col(classes).n_unique()).item()
    LIMITS = {"family": 60, "genus": 128, "species": 927}
    full   = pl.DataFrame({classes: range(1, LIMITS[classes] + 1)})
    counts = df.group_by(classes).agg(pl.len().alias("n"))
    joined = full.join(counts, on=classes, how="left")
    missing = joined.filter(pl.col("n").is_null()).get_column(classes).to_list()
    dev = (
        joined.with_columns((pl.col("n").fill_null(0) - exp_even).alias("dev"))
            .sort(classes)
    )
    plt.figure(figsize=(11,4))
    plt.bar(dev.get_column(classes).to_list(), dev["dev"].to_list(), width=0.9, label="Actual − Expected")
    plt.axhline(0, linewidth=1)
    handle = Line2D([0],[0], color='none')
    label  = f"Missing {classes}: " + (", ".join(map(str, missing)) if missing else "None")
    plt.legend([handle], [label], loc='center left', bbox_to_anchor=(1.0, 0.5), frameon=False)
    N = LIMITS[classes]
    step = 5 if N <= 60 else (10 if N <= 150 else 50)
    plt.xticks(range(step, N + 1, step))

    plt.xlabel(classes.capitalize()); plt.ylabel("Actual − Expected")
    plt.title(f"Deviation per {classes.capitalize()} (expected ≈ {exp_even:.2f}/{classes})")
    plt.tight_layout()
    plt.show()

In [ ]:
for c in classes:
    data = pl.read_parquet(DATA)
    distribution_vs_expected(data, c)

In [ ]:
import gc 
gc.collect()

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from pathlib import Path

In [ ]:
spec = data.iloc[:, 2].to_numpy()
u, inv = np.unique(spec, return_inverse=True)
cnt = np.bincount(inv)
x1 = np.log1p(cnt)
x2 = np.argsort(np.argsort(cnt)) / (len(cnt) - 1 + 1e-9)
X_all = np.c_[x1, x2]

k = 2
TESTSIZE = 0.4

km = KMeans(n_clusters=k, n_init=10, random_state=0).fit(x1.reshape(-1,1))
order = np.argsort(km.cluster_centers_.ravel())
y_all = np.empty_like(km.labels_)
for r, c in enumerate(order): y_all[km.labels_ == c] = r
X, ins, y, outs = train_test_split(
    X_all, y_all, test_size=TESTSIZE, stratify=y_all, random_state=0
)

scaler = StandardScaler()
X = scaler.fit_transform(X)
ins = scaler.transform(ins)
y, outs = y, outs

KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(X, y)
xpred   = KNN.predict(X)
outpred = KNN.predict(ins)
print("Train:", accuracy_score(y, xpred))
print("Test :", accuracy_score(outs, outpred))

In [ ]:
conf = confusion_matrix(outs, outpred)
conf

In [ ]:
x_min, x_max = X[:,0].min()-0.5, X[:,0].max()+0.5
y_min, y_max = X[:,1].min()-0.5, X[:,1].max()+0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))
Z = KNN.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.figure(figsize=FIGSIZE)
plt.contourf(xx, yy, Z, alpha=0.25)
plt.scatter(X[:,0], X[:,1], c=y, s=10, edgecolors='k')
plt.scatter(ins[:,0], ins[:,1], c=outs, s=10, edgecolors='r')
plt.title("KNN rarity (0=rarest)")
plt.xlabel("log1p(count)") 
plt.ylabel("species frequency")

In [ ]:
import gc 
gc.collect()

# End‑to‑End Pipeline: Stage 1 (SVM Gate) → Stage 2 (RF/Bags + Stacker) → Stage 3 (Boosted Finaliser)

This notebook runs your full classification pipeline in three stages:

- **Stage 1**: SVM rarity gate (GMM/KMeans/KNN frequency split) with ROC/TPR‑FPR plots and threshold sweep.
- **Stage 2**: Random‑Forest/Subag/Bagging base models per level (Family & Genus) + a Logistic stacker with meta‑features (entropy, margin, optional gate prob). Includes confusion matrix, ROC and PR plots.
- **Stage 3**: Final boosted classifier (AdaBoost, optionally XGBoost if present) on meta‑features from Stage 2 (family/genus stacker probabilities + p_rare).

> Adjust the `DATA` path below to point at your parquet with columns `family`, `genus`, `species` and numeric feature columns.


In [ ]:
# === Imports & config ===
import gc; gc.collect()
import time
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from joblib import dump
from joblib import load
from collections import Counter

# sklearn
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Optional XGBoost (only if installed)
try:
    from xgboost import XGBClassifier
    XGB_AVAILABLE = True
except Exception:
    XGB_AVAILABLE = False

# Style / constants
RS = 1234
K  = 5
FIGSIZE = (10, 6)

# DATA path: update this to your parquet file
DATA = Path("Data/master.parquet")

# Output dirs
PLOTS = Path("Plots"); PLOTS.mkdir(parents=True, exist_ok=True)
GATE = Path("gate"); GATE.mkdir(parents = True, exist_ok = True)
PLOTS_ROC = PLOTS / "ROC";        PLOTS_ROC.mkdir(parents=True, exist_ok=True)
PLOTS_RF  = PLOTS / "RF";         PLOTS_RF.mkdir(parents=True, exist_ok=True)
PLOTS_BND = PLOTS                 # for threshold sweeps
MODELS = Path("Models");          MODELS.mkdir(parents=True, exist_ok=True)
MODELS_SVM = MODELS / "SVM";      MODELS_SVM.mkdir(parents=True, exist_ok=True)
MODELS_RF  = MODELS / "RF";       MODELS_RF.mkdir(parents=True, exist_ok=True)
MODELS_META= MODELS / "Stage2Meta"; MODELS_META.mkdir(parents=True, exist_ok=True)
OUTPUT = Path("output");          OUTPUT.mkdir(parents=True, exist_ok=True)

# Utility plotting note: Jupyter inline
# %matplotlib inline  # (uncomment if you run in classic Notebook)


In [ ]:
# === Helpers: data loading, rarity labels, metrics, utilities ===

def load_xy(drop_cols=("family","genus","species")):
    """Load parquet data, return (data, feature_matrix).
    Expects columns family/genus/species + numeric features.
    """
    data = pd.read_parquet(DATA)
    X = data.drop(columns=list(drop_cols), errors="ignore").to_numpy(np.float32)
    return data, X


def initial_rare_labels(species, method="kmeans", random_state=RS, save_model=True, return_counts=False):
    """Compute rarity labels (hard + soft) from class frequencies via GMM/KMeans/KNN proxy."""
    cnt = Counter(species)
    counts = np.fromiter((cnt[s] for s in species), dtype=np.int32, count=len(species))
    x = np.log1p(counts).reshape(-1, 1)

    if method == "gmm":
        model = GaussianMixture(n_components=2, random_state=random_state).fit(x)
        rare_comp = np.argmin(model.means_.ravel())
        hard = (model.predict(x) == rare_comp).astype(np.int32)
        soft = model.predict_proba(x)[:, rare_comp]
        if save_model: dump(model, MODELS / "01_species_gmm.joblib")

    elif method == "kmeans":
        model = KMeans(n_clusters=2, n_init=10, random_state=random_state).fit(x)
        rare_comp = np.argmin(model.cluster_centers_.ravel())
        hard = (model.labels_ == rare_comp).astype(np.int32)
        d = np.linalg.norm(x - model.cluster_centers_[rare_comp], axis=1)
        soft = (d.max() - d) / (d.max() - d.min() + 1e-9)
        if save_model: dump(model, MODELS / "01_species_kmeans.joblib")

    elif method == "knn":
        k = 10
        model = KNeighborsRegressor(n_neighbors=k)
        model.fit(x, counts)
        local_density = model.predict(x)
        soft = 1 - (local_density - local_density.min()) / (local_density.max() - local_density.min() + 1e-9)
        hard = (soft > np.median(soft)).astype(np.int32)
        if save_model: dump(model, MODELS / "01_species_knn.joblib")

    else:
        raise ValueError("method must be 'gmm', 'kmeans', or 'knn'")

    return (hard, soft, counts) if return_counts else (hard, soft)


def threshold(y_true, y_proba, thr=0.5):
    """Compute TP,TN,FP,FN for a given threshold on positive class probability."""
    # supports y_proba being 1D or 2D
    p = y_proba[:,1] if y_proba.ndim == 2 else y_proba
    pred = (p >= thr).astype(int)
    cm = confusion_matrix(y_true, pred, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    return int(tp), int(tn), int(fp), int(fn)


def binary_metrics(y_true, y_proba, thr, csv_path=None, classifier_name="", method="", train_size=None):
    tp, tn, fp, fn = threshold(y_true, y_proba, thr)
    acc = (tp+tn) / max(tp+tn+fp+fn, 1)
    prec = tp / max(tp+fp, 1)
    rec = tp / max(tp+fn, 1)
    from math import sqrt
    f1 = 2*prec*rec / max(prec+rec, 1e-12)
    if csv_path is not None:
        recrow = {
            "classifier": classifier_name,
            "method": method,
            "train_size": train_size,
            "threshold": thr,
            "tp": tp, "tn": tn, "fp": fp, "fn": fn,
            "acc": acc, "precision": prec, "recall": rec, "f1": f1,
        }
        try:
            df = pd.DataFrame([recrow])
            if Path(csv_path).exists():
                df.to_csv(csv_path, mode="a", header=False, index=False)
            else:
                df.to_csv(csv_path, index=False)
        except Exception:
            pass
    return acc, prec, rec, f1


def entropy_row(p):
    p = np.clip(p, 1e-12, 1.0)
    return -(p * np.log(p)).sum(axis=1)


def top_margin(p):
    s = np.sort(p, axis=1)[:, ::-1]
    return s[:,0] - s[:,1]


def hard_vote(preds_list):
    preds = np.stack(preds_list, axis=1)
    out = []
    for row in preds:
        vals, cnts = np.unique(row, return_counts=True)
        out.append(vals[np.argmax(cnts)])
    return np.array(out)


def soft_vote(probas_list):
    P = np.mean(probas_list, axis=0)
    return P, P.argmax(axis=1)


def oracle_bound(preds_list, y_true):
    preds = np.stack(preds_list, axis=1)
    ok = (preds == y_true[:, None]).any(axis=1)
    return ok.mean()


def plot_confusion(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred, normalize="true")
    fig, ax = plt.subplots(figsize=(7,6))
    im = ax.imshow(cm, aspect='auto')
    ax.set_title(title + "\nNormalized Confusion Matrix")
    ax.set_xlabel("Predicted"); ax.set_ylabel("Actual")
    fig.colorbar(im, ax=ax)
    plt.show()


def plot_roc(y_true, y_score, n_classes, title):
    y_true_bin = pd.get_dummies(y_true)
    fig, ax = plt.subplots(figsize=(7,6))
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_true_bin.iloc[:, i], y_score[:, i])
        auc = roc_auc_score(y_true_bin.iloc[:, i], y_score[:, i])
        ax.plot(fpr, tpr, label=f"Class {i} (AUC={auc:.2f})")
    ax.plot([0,1],[0,1],'--')
    ax.set_title("ROC – " + title); ax.set_xlabel("FPR"); ax.set_ylabel("TPR")
    ax.legend()
    plt.show()


def plot_pr(y_true, y_score, n_classes, title):
    y_true_bin = pd.get_dummies(y_true)
    fig, ax = plt.subplots(figsize=(7,6))
    for i in range(n_classes):
        pr, rc, _ = precision_recall_curve(y_true_bin.iloc[:, i], y_score[:, i])
        ax.plot(rc, pr, label=f"Class {i}")
    ax.set_title("Precision–Recall – " + title); ax.set_xlabel("Recall"); ax.set_ylabel("Precision")
    ax.legend()
    plt.show()


## Stage 1 — SVM Rarity Gate

In [ ]:
# === Stage 1: SVM definitions ===

# def SVMLIN():
#     clf = Pipeline([
#         ("mm", MinMaxScaler()),
#         ("svc", SVC(kernel="linear", probability=True, tol=1e-2, max_iter=1500, decision_function_shape="ovr", random_state=RS))
#     ])
#     clf.name = "SVMLIN"; return clf

def SVMRBF():
    clf = Pipeline([
        ("mm", MinMaxScaler()),
        ("svc", SVC(kernel="rbf", probability=True, tol=1e-2, max_iter=1500, decision_function_shape="ovr", random_state=RS))
    ])
    clf.name = "SVMRBF"; return clf

# def SVMPoly(d=2):
#     clf = Pipeline([
#         ("mm", MinMaxScaler()),
#         ("svc", SVC(kernel="poly", degree=d, probability=True, tol=1e-2, max_iter=1500, decision_function_shape="ovr", random_state=RS))
#     ])
#     clf.name = f"SVMPoly{d}"; return clf


In [ ]:
# === Stage 1: Trainer ===

def SVM_TRAIN_LOG(CLASSIFIER, METHOD, TRAINSIZE, THRESHOLD=0.5):
    j = getattr(CLASSIFIER, "name", CLASSIFIER.__class__.__name__)
    print(f"Classifier name: {j}")

    data, X = load_xy()
    species = data["species"].to_numpy()
    y, p_rare = initial_rare_labels(species, method=METHOD)

    TESTSIZE = 1 - (0.01 * TRAINSIZE)
    X, ins, y, outs = train_test_split(X, y, test_size=TESTSIZE, stratify=y, random_state=RS)

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RS)
    _ = cross_val_predict(CLASSIFIER, X, y, cv=cv, method="predict", n_jobs=1)

    CLASSIFIER.fit(X, y)
    proba = CLASSIFIER.predict_proba(ins)

    # paths
    ROC1 = PLOTS_ROC / f"{METHOD}_{j}_{TRAINSIZE}_roc.png"
    ROC2 = PLOTS_ROC / f"{METHOD}_{j}_{TRAINSIZE}_tprfpr.png"
    mpath = MODELS_SVM / f"{METHOD}_{j}_{TRAINSIZE}.joblib"
    plot_path = PLOTS_BND / f"{METHOD}_{j}_{TRAINSIZE}_thresholds.png"

    # save model
    dump(CLASSIFIER, mpath)

    # metrics + plots
    binary_metrics(outs, proba, THRESHOLD,
                   csv_path= OUTPUT / "all_metrics.csv",
                   classifier_name=j, method=METHOD, train_size=TRAINSIZE)

    thresholds = np.linspace(0.1, 0.9, 1001)
    TP, TN, FP, FN = [], [], [], []
    for thr in thresholds:
        tp, tn, fp, fn = threshold(outs, proba, thr)
        TP.append(tp); TN.append(tn); FP.append(fp); FN.append(fn)

    fig, ax = plt.subplots(figsize=FIGSIZE)
    ax.plot(thresholds, TP, label="TP")
    ax.plot(thresholds, TN, label="TN")
    ax.plot(thresholds, FP, label="FP")
    ax.plot(thresholds, FN, label="FN")
    ax.set_xlim([0.1, 0.9])
    ax.set_xlabel("Threshold"); ax.set_ylabel("Count"); ax.legend()
    fig.savefig(plot_path, dpi=300, bbox_inches="tight"); plt.close(fig)

    # ROC bits (use pos class prob)
    if proba.ndim > 1:
        proba_pos = proba[:,1]
    else:
        proba_pos = proba

    fpr, tpr, roc_thresholds = roc_curve(outs, proba_pos)
    auc_score = roc_auc_score(outs, proba_pos)

    fig, ax = plt.subplots(figsize=FIGSIZE)
    ax.plot(roc_thresholds, tpr, label="TPR")
    ax.plot(roc_thresholds, fpr, label="FPR")
    ax.plot(roc_thresholds, tpr - fpr, label="TPR-FPR")
    ax.invert_xaxis(); ax.set_xlabel("Threshold"); ax.set_ylabel("Fraction"); ax.legend(); ax.grid(alpha=0.3)
    fig.savefig(ROC2, dpi=300, bbox_inches="tight"); plt.close(fig)

    fig, ax = plt.subplots(figsize=FIGSIZE)
    ax.plot(fpr, tpr, lw=2, label=f"ROC (AUC={auc_score:.3f})")
    ax.plot([0,1],[0,1],'--', lw=1, label="Random")
    ax.set_xlabel("FPR"); ax.set_ylabel("TPR"); ax.set_xlim([0,1]); ax.set_ylim([0,1.05]); ax.set_aspect('equal','box'); ax.legend(); ax.grid(alpha=0.3)
    fig.savefig(ROC1, dpi=300, bbox_inches="tight"); plt.close(fig)

    print(f"✅ Done: {METHOD} – {j} – {TRAINSIZE}")
    return (METHOD, j, TRAINSIZE)


In [ ]:
# === Stage 1: Run (adjust to taste) ===
from itertools import product
from concurrent.futures import ThreadPoolExecutor, as_completed

FIGSIZE = (10,6)
THRESHOLD = 0.501
TRAINS = [75]  # e.g., 75% train inside gate
methods = ["gmm", "kmeans", "knn"]
# classifiers = [SVMLIN(), SVMRBF(), SVMPoly(2), SVMPoly(3), SVMPoly(4), SVMPoly(5)]
classifiers = [SVMRBF()]

start = time.perf_counter()
tasks = list(product(classifiers, methods, TRAINS))

# You can reduce workers if CPU is tight
with ThreadPoolExecutor(max_workers=4) as ex:
    futures = [ex.submit(SVM_TRAIN_LOG, *args, THRESHOLD=THRESHOLD) for args in tasks]
    for f in as_completed(futures):
        print(f.result())

end = time.perf_counter(); print(f"\n⏱ Stage 1 time: {end - start:.2f} sec")


## Stage 2 — RF/Subag/Bagging + Stacker (Family & Genus)

In [ ]:
# === Stage 2: train meta-ensemble utilities ===

def train_meta_ensemble(X, y, ins, outs, base_models, p_rare=None, label="family"):
    n_classes = len(np.unique(y))
    cv = StratifiedKFold(n_splits=K, shuffle=True, random_state=RS)

    oof_list, preds_list, prob_list = [], [], []
    print(f"\nTraining base models for {label.upper()} ...")

    for name, est in base_models:
        oof = cross_val_predict(est, X, y, cv=cv, method="predict_proba", n_jobs=-1)
        oof_list.append(oof)
        est.fit(X, y)
        p = est.predict(ins)
        P = est.predict_proba(ins)
        preds_list.append(p); prob_list.append(P)
        print(f"  {name:<4} acc = {accuracy_score(outs, p):.4f}")

    # Build stacker matrices
    stack_train = np.hstack(oof_list)
    stack_test  = np.hstack(prob_list)

    stack_train = np.hstack([
        stack_train,
        entropy_row(np.mean(oof_list, axis=0)).reshape(-1,1),
        top_margin(np.mean(oof_list, axis=0)).reshape(-1,1)
    ])
    add_test = [
        entropy_row(np.mean(prob_list, axis=0)).reshape(-1,1),
        top_margin(np.mean(prob_list, axis=0)).reshape(-1,1)
    ]
    if p_rare is not None:
        add_test.append(p_rare[:len(ins)].reshape(-1,1))
    stack_test = np.hstack([stack_test] + add_test)

    stacker = LogisticRegression(max_iter=200, n_jobs=-1, random_state=RS)
    stacker.fit(stack_train, y)
    y_pred_stack  = stacker.predict(ins)
    y_proba_stack = stacker.predict_proba(ins)

    # metrics
    acc_vote  = accuracy_score(outs, hard_vote(preds_list))
    acc_soft  = accuracy_score(outs, soft_vote(prob_list)[1])
    acc_stack = accuracy_score(outs, y_pred_stack)
    acc_oracle = oracle_bound(preds_list, outs)

    print(f"\nResults – {label.upper()}")
    print(f"  Hard vote  : {acc_vote:.4f}")
    print(f"  Soft vote  : {acc_soft:.4f}")
    print(f"  Stacker    : {acc_stack:.4f}")
    print(f"  Oracle     : {acc_oracle:.4f}\n")
    print(classification_report(outs, y_pred_stack, digits=3))

    plot_confusion(outs, y_pred_stack, f"{label.upper()} – Stacker")
    plot_roc(outs, y_proba_stack, n_classes, f"{label.upper()} – Stacker")
    plot_pr(outs, y_proba_stack, n_classes, f"{label.upper()} – Stacker")

    return stacker, y_pred_stack, y_proba_stack


def build_mapping_family_to_genus(family, genus):
    mapping = {}
    for f, g in zip(family, genus):
        mapping.setdefault(int(f), set()).add(int(g))
    return mapping


def enforce_hierarchy(pred_family, proba_genus, mapping):
    fixed = proba_genus.copy()
    for i, f in enumerate(pred_family):
        allowed = mapping.get(int(f), None)
        if allowed is None:
            continue
        mask = np.ones(fixed.shape[1], dtype=bool)
        mask[list(allowed)] = False
        fixed[i, mask] = 0.0
        s = fixed[i].sum()
        if s > 0:
            fixed[i] /= s
    return fixed, fixed.argmax(axis=1)


In [ ]:
# === Stage 2: run ===
# Load data once for Stage 2
data, X = load_xy()
species = data["species"].to_numpy()
y_family = data["family"].to_numpy()
y_genus  = data["genus"].to_numpy()

method = "kmeans"  # or "gmm" / "knn" depending on which you used

if method == "kmeans":
    rarity_model = load(MODELS / "01_species_kmeans.joblib")
    x = np.log1p(np.bincount(species))
    x = np.log1p(np.array([np.count_nonzero(species == s) for s in species])).reshape(-1, 1)
    rare_comp = np.argmin(rarity_model.cluster_centers_.ravel())
    d = np.linalg.norm(x - rarity_model.cluster_centers_[rare_comp], axis=1)
    p_rare_full = (d.max() - d) / (d.max() - d.min() + 1e-9)

elif method == "gmm":
    rarity_model = load(MODELS / "01_species_gmm.joblib")
    x = np.log1p(np.array([np.count_nonzero(species == s) for s in species])).reshape(-1, 1)
    rare_comp = np.argmin(rarity_model.means_.ravel())
    p_rare_full = rarity_model.predict_proba(x)[:, rare_comp]

elif method == "knn":
    rarity_model = load(MODELS / "01_species_knn.joblib")
    x = np.log1p(np.array([np.count_nonzero(species == s) for s in species])).reshape(-1, 1)
    local_density = rarity_model.predict(x)
    p_rare_full = 1 - (local_density - local_density.min()) / (local_density.max() - local_density.min() + 1e-9)

else:
    raise ValueError("Unsupported method for rarity model")

print("✅ Loaded rarity model from Stage 1")
# Keep Stage-1 convention: X/ins per target
X_fam, ins_fam, yF, outsF = train_test_split(_X, y_family, test_size=0.2, stratify=y_family, random_state=RS)
X_gen, ins_gen, yG, outsG = train_test_split(_X, y_genus,  test_size=0.2, stratify=y_genus,  random_state=RS)

base_family = [
    ("RF",  RandomForestClassifier(n_estimators=300, bootstrap=True,  n_jobs=-1, random_state=RS)),
    ("Sub", RandomForestClassifier(n_estimators=300, bootstrap=False, n_jobs=-1, random_state=RS)),
    ("Bag", BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RS), n_estimators=200, bootstrap=True, n_jobs=-1, random_state=RS)),
]
base_genus = [
    ("RF",  RandomForestClassifier(n_estimators=400, bootstrap=True,  n_jobs=-1, random_state=RS)),
    ("Sub", RandomForestClassifier(n_estimators=400, bootstrap=False, n_jobs=-1, random_state=RS)),
    ("Bag", BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RS), n_estimators=300, bootstrap=True, n_jobs=-1, random_state=RS)),
]

start2 = time.perf_counter()
print("Training FAMILY (no gate prob in stacker test)")
fam_stacker, fam_pred, fam_proba = train_meta_ensemble(X_fam, yF, ins_fam, outsF, base_family, p_rare=None, label="family")

print("\nTraining GENUS (with gate prob in stacker test)")
gen_stacker, gen_pred, gen_proba = train_meta_ensemble(X_gen, yG, ins_gen, outsG, base_genus, p_rare=p_rare_full, label="genus")

print(f"\n✅ Stage 2 finished in {time.perf_counter()-start2:.1f}s")

# Enforce hierarchy
fam2gen = build_mapping_family_to_genus(y_family, y_genus)
gen_fixed_proba, gen_fixed_pred = enforce_hierarchy(fam_pred, gen_proba, fam2gen)
print(f"[Hierarchy] genus corrections {(gen_fixed_pred != gen_pred).mean():.3f} of samples changed.")
print(f"[Hierarchy] corrected genus acc = {accuracy_score(outsG, gen_fixed_pred):.4f}")
plot_confusion(outsG, gen_fixed_pred, "GENUS – Hierarchy‑Corrected")


## Stage 3 — Final Boosted Classifier (AdaBoost / XGBoost if available)

In [ ]:
# === Stage 3: build meta-features and train boosted finaliser (Updated) ===
# Build species target, align lengths conservatively
_data2, _X2 = load_xy()
species_all = _data2["species"].to_numpy()

n = min(len(p_rare_full), len(fam_proba), len(gen_proba))
meta3_X = np.hstack([
    p_rare_full[:n].reshape(-1, 1),
    fam_proba[:n],
    gen_proba[:n],
])
meta3_y = species_all[:n]

X3, X3_test, y3, y3_test = train_test_split(
    meta3_X, meta3_y, test_size=0.2,
    stratify=meta3_y, random_state=RS
)

# === AdaBoost (modern sklearn) ===
# Default algorithm='SAMME' is the only supported mode now.
# 'SAMME' performs discrete boosting, perfectly fine for multi-class classification.
ada = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=3, random_state=RS),
    n_estimators=400,
    learning_rate=0.8,
    algorithm="SAMME",
    random_state=RS
)
ada.fit(X3, y3)
ada_pred  = ada.predict(X3_test)
ada_proba = ada.predict_proba(X3_test)

print("\n[Stage 3 – AdaBoost]")
print(classification_report(y3_test, ada_pred, digits=3))
print("Accuracy:", accuracy_score(y3_test, ada_pred))

# --- Confusion matrix for AdaBoost ---
cm = confusion_matrix(y3_test, ada_pred, normalize="true")
fig, ax = plt.subplots(figsize=(8,6))
im = ax.imshow(cm, aspect='auto')
ax.set_title("Stage 3 – AdaBoost Final Classifier\nNormalized Confusion Matrix")
ax.set_xlabel("Predicted"); ax.set_ylabel("True")
fig.colorbar(im, ax=ax)
plt.show()

# === Optional XGBoost comparison ===
if XGB_AVAILABLE:
    xgb = XGBClassifier(
        n_estimators=500,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="mlogloss",
        random_state=RS,
        n_jobs=-1
    )
    xgb.fit(X3, y3)
    xgb_pred  = xgb.predict(X3_test)
    xgb_proba = xgb.predict_proba(X3_test)
    print("\n[Stage 3 – XGBoost]")
    print(classification_report(y3_test, xgb_pred, digits=3))
    print("Accuracy:", accuracy_score(y3_test, xgb_pred))

    cm2 = confusion_matrix(y3_test, xgb_pred, normalize="true")
    fig, ax = plt.subplots(figsize=(8,6))
    im = ax.imshow(cm2, aspect='auto')
    ax.set_title("Stage 3 – XGBoost Final Classifier\nNormalized Confusion Matrix")
    ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    fig.colorbar(im, ax=ax)
    plt.show()
else:
    print("\n[XGBoost not available] Skipping XGB comparison.")


In [ ]:
from joblib import load

def run_inference(new_data_path: str):
    """
    Run the full 3-stage pipeline on a new dataset (no labels required).
    Expects same numeric columns as training data.
    """
    print(f"\n🔍 Loading new data from: {new_data_path}")
    df_new = pd.read_parquet(new_data_path)
    X_new = df_new.drop(columns=["family", "genus", "species"], errors="ignore").to_numpy(np.float32)

    # --- Stage 1: rarity gate ---
    print("Stage 1: rarity gate (predict p_rare)...")
    gmm_model = load(MODELS / "01_species_kmeans.joblib")  # or 01_species_gmm.joblib depending on what you used
    counts = np.log1p(np.ones(len(X_new)))  # dummy shape alignment
    d = np.linalg.norm(counts.reshape(-1, 1) - gmm_model.cluster_centers_[np.argmin(gmm_model.cluster_centers_)], axis=1)
    p_rare = (d.max() - d) / (d.max() - d.min() + 1e-9)

    # --- Stage 2: family + genus prediction ---
    print("Stage 2: predicting family & genus via stackers...")
    fam_stacker = load(MODELS_META / "family_stacker.joblib")
    gen_stacker = load(MODELS_META / "genus_stacker.joblib")
    base_fam_models = [
        load(MODELS_RF / "family_RF.joblib"),
        load(MODELS_RF / "family_Sub.joblib"),
        load(MODELS_RF / "family_Bag.joblib"),
    ]
    base_gen_models = [
        load(MODELS_RF / "genus_RF.joblib"),
        load(MODELS_RF / "genus_Sub.joblib"),
        load(MODELS_RF / "genus_Bag.joblib"),
    ]

    fam_prob_list = [m.predict_proba(X_new) for m in base_fam_models]
    gen_prob_list = [m.predict_proba(X_new) for m in base_gen_models]
    fam_probs_mean = np.mean(fam_prob_list, axis=0)
    gen_probs_mean = np.mean(gen_prob_list, axis=0)

    fam_entropy = -(fam_probs_mean * np.log(fam_probs_mean + 1e-9)).sum(axis=1)
    fam_margin = np.sort(fam_probs_mean, axis=1)[:, -1] - np.sort(fam_probs_mean, axis=1)[:, -2]
    gen_entropy = -(gen_probs_mean * np.log(gen_probs_mean + 1e-9)).sum(axis=1)
    gen_margin = np.sort(gen_probs_mean, axis=1)[:, -1] - np.sort(gen_probs_mean, axis=1)[:, -2]

    fam_meta = np.hstack([fam_probs_mean, fam_entropy.reshape(-1, 1), fam_margin.reshape(-1, 1), p_rare.reshape(-1, 1)])
    gen_meta = np.hstack([gen_probs_mean, gen_entropy.reshape(-1, 1), gen_margin.reshape(-1, 1), p_rare.reshape(-1, 1)])

    fam_pred = fam_stacker.predict(fam_meta)
    gen_pred = gen_stacker.predict(gen_meta)
    fam_proba = fam_stacker.predict_proba(fam_meta)
    gen_proba = gen_stacker.predict_proba(gen_meta)

    # --- Stage 3: final boosted classifier ---
    print("Stage 3: final boosted classifier (AdaBoost / XGB)...")
    booster_path = MODELS / "Stage3_Booster.joblib"
    if booster_path.exists():
        booster = load(booster_path)
    else:
        raise FileNotFoundError("Stage 3 booster model not found!")

    meta_final = np.hstack([p_rare.reshape(-1, 1), fam_proba, gen_proba])
    species_pred = booster.predict(meta_final)
    species_proba = booster.predict_proba(meta_final)

    # --- Combine outputs ---
    results = pd.DataFrame({
        "family_pred": fam_pred,
        "genus_pred": gen_pred,
        "species_pred": species_pred,
        "p_rare": p_rare,
        "entropy_family": fam_entropy,
        "entropy_genus": gen_entropy,
        "margin_family": fam_margin,
        "margin_genus": gen_margin,
    })
    print(f"\n✅ Inference complete: {len(results)} samples processed.")
    out_path = Path("output/predictions_new.csv")
    results.to_csv(out_path, index=False)
    print(f"Saved predictions → {out_path.resolve()}")

    return results

# Test on each of the original files:


In [ ]:
def batch_inference(data_dir="Data"):
    data_dir = Path(data_dir)
    parquet_files = [p for p in data_dir.glob("*.parquet") if p.name.lower() != "master.parquet"]

    if not parquet_files:
        print(f"⚠️ No new parquet files found in {data_dir}")
        return

    print(f"🧩 Found {len(parquet_files)} parquet file(s) for inference:")
    for f in parquet_files:
        print(f"  - {f.name}")

    # Load models once
    booster_path = MODELS / "Stage3_Booster.joblib"
    if not booster_path.exists():
        raise FileNotFoundError("❌ Stage 3 booster not found!")

    booster = load(booster_path)
    fam_stacker = load(MODELS_META / "family_stacker.joblib")
    gen_stacker = load(MODELS_META / "genus_stacker.joblib")

    base_fam_models = [
        load(MODELS_RF / "family_RF.joblib"),
        load(MODELS_RF / "family_Sub.joblib"),
        load(MODELS_RF / "family_Bag.joblib"),
    ]
    base_gen_models = [
        load(MODELS_RF / "genus_RF.joblib"),
        load(MODELS_RF / "genus_Sub.joblib"),
        load(MODELS_RF / "genus_Bag.joblib"),
    ]

    # Load rarity model
    rarity_model_path = MODELS / "01_species_kmeans.joblib"
    rarity_model = load(rarity_model_path)

    for f in parquet_files:
        print(f"\n🚀 Running inference on {f.name} ...")

        df = pd.read_parquet(f)
        X = df.drop(columns=["family", "genus", "species"], errors="ignore").to_numpy(np.float32)

        # Stage 1: rarity gate
        d = np.linalg.norm(
            np.log1p(np.ones(len(X))).reshape(-1, 1)
            - rarity_model.cluster_centers_[np.argmin(rarity_model.cluster_centers_)],
            axis=1
        )
        p_rare = (d.max() - d) / (d.max() - d.min() + 1e-9)

        # Stage 2: base + meta predictions
        fam_prob_list = [m.predict_proba(X) for m in base_fam_models]
        gen_prob_list = [m.predict_proba(X) for m in base_gen_models]

        fam_probs_mean = np.mean(fam_prob_list, axis=0)
        gen_probs_mean = np.mean(gen_prob_list, axis=0)

        fam_entropy = -(fam_probs_mean * np.log(fam_probs_mean + 1e-9)).sum(axis=1)
        fam_margin = np.sort(fam_probs_mean, axis=1)[:, -1] - np.sort(fam_probs_mean, axis=1)[:, -2]
        gen_entropy = -(gen_probs_mean * np.log(gen_probs_mean + 1e-9)).sum(axis=1)
        gen_margin = np.sort(gen_probs_mean, axis=1)[:, -1] - np.sort(gen_probs_mean, axis=1)[:, -2]

        fam_meta = np.hstack([
            fam_probs_mean,
            fam_entropy.reshape(-1, 1),
            fam_margin.reshape(-1, 1),
            p_rare.reshape(-1, 1)
        ])
        gen_meta = np.hstack([
            gen_probs_mean,
            gen_entropy.reshape(-1, 1),
            gen_margin.reshape(-1, 1),
            p_rare.reshape(-1, 1)
        ])

        fam_pred = fam_stacker.predict(fam_meta)
        gen_pred = gen_stacker.predict(gen_meta)
        fam_proba = fam_stacker.predict_proba(fam_meta)
        gen_proba = gen_stacker.predict_proba(gen_meta)

        # Stage 3: booster
        meta_final = np.hstack([p_rare.reshape(-1, 1), fam_proba, gen_proba])
        species_pred = booster.predict(meta_final)

        out_df = df.copy()
        out_df["family_pred"] = fam_pred
        out_df["genus_pred"] = gen_pred
        out_df["species_pred"] = species_pred
        out_df["p_rare"] = p_rare
        out_df["entropy_family"] = fam_entropy
        out_df["entropy_genus"] = gen_entropy
        out_df["margin_family"] = fam_margin
        out_df["margin_genus"] = gen_margin

        out_path = OUTPUT / f"pred_{f.stem}.csv"
        out_df.to_csv(out_path, index=False)
        print(f"✅ Saved predictions → {out_path.resolve()}")

    print("\n🏁 All parquet files processed.\n")

# Run it
batch_inference("Data")
